# Photoevaporation of a dense clump

## TOC
* [Packages & Settings](#packages)
* [Snapshots](#snapshots)
* [Loading Snapshots](#loading)
* [Maps](#maps)
* [Lineouts](#lineouts)
* [I-front position](#pos-vel)

## Packages & Settings  <a class="anchor" id="packages"></a>

In [ ]:
from pyrhyme import PyRhyme

from pathlib import Path

# !pip install numpy
import numpy as np

# !pip install astropy
from astropy import units as U
from astropy import constants as C
from astropy.cosmology import WMAP7

# !pip install scipy
from scipy.io import FortranFile
from scipy.interpolate import interp1d

# !pip install plotly
import plotly
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
from plotly.subplots import make_subplots
pio.templates.default = "simple_white"

FIGURE_DIR = './figures'
Path(FIGURE_DIR).mkdir(parents=True, exist_ok=True)

## Snapshots  <a class="anchor" id="snapshots"></a>

Run
```bash
wget -r –level=0 -E –ignore-length -x -k -p -erobots=off -np -N https://astronomy.sussex.ac.uk/\~iti20/RT_comparison_project/RT_workshop_data/T7_results/
```
to download Iliev-7 test snapshots!

In [ ]:
ILIEV_7_DIR = 'astronomy.sussex.ac.uk/~iti20/RT_comparison_project/RT_workshop_data/T7_results'

TIMES = [1, 3, 10, 25, 75]

SIMULATION_NAMES = [
    ('CAPREOLE+C2-RAY', 'Mellema'),
    ('CORAL', 'coral'),
    ('FLASH-HC', 'flash'),
    ('LICORICE', 'Licorice'),
    ('RSPH', 'Susa'),
    ('ZEUS-MP', 'Zeus'),
]

BINARIES = {sn: {
    'orig': {t: {'path': f"{ILIEV_7_DIR}/{abbr}{ti+1}.bin"} for ti, t in enumerate(TIMES)},
    'add': {t: {'path': f"{ILIEV_7_DIR}/{abbr}_add{ti+1}.bin"} for ti, t in enumerate(TIMES)}
} for sn, abbr in SIMULATION_NAMES}


RHYME = {
    'Rhyme Case A': {
        0: {'path': './CaseA/Iliev-7-CaseA-000001.chombo.h5'},
    },
    'Rhyme Case B': {
        0: {'path': './CaseB/Iliev-7-CaseB-000001.chombo.h5'}
    }
}

# Check if files exist
def check_if_exist(path):
    if not Path(snap['path']).is_file():
        print(f"Not found: {snap['path']}")
        
for sim_name, sim in BINARIES.items():      
    for snap in sim['orig'].values():
        check_if_exist(snap['path'])
    for snap in sim['add'].values():
        check_if_exist(snap['path'])

for sim in RHYME.values():
    check_if_exist(sim[0]['path'])

## Loading snapshots  <a class="anchor" id="loading"></a>

In [ ]:
import os
def time_to_snap_id(t, ds):
    for i in range(0, ds.dataset.num_of_snapshots):
        ds.dataset.jump_to(i)
        snap_time = ds.dataset.time
        if snap_time > t:
            if i == 0:
                return i
            
            ds.dataset.jump_to(i-1)
            prev_time = ds.dataset.time
            
            return i if abs(t - snap_time) < abs(t - prev_time) else i - 1
        
    return ds.dataset.num_of_snapshots - 1


def load_rhyme(snap_id, rhyme, Gamma=5./3.):
    rhyme.dataset.jump_to(snap_id)
    res = {}

    v = rhyme.load_variables(silent=True)
    domain = rhyme.dataset.problem_domain

    rho = v['rho'][0].reshape(domain) * (1 * v['rho'][1]).to(U.cm**-3).value
    vx = v['rho_u'][0].reshape(domain) / rho * (1 * U.Mpc / U.Myr).to(U.cm / U.s).value
    vy = v['rho_v'][0].reshape(domain) / rho * (1 * U.Mpc / U.Myr).to(U.cm / U.s).value
    vz = v['rho_w'][0].reshape(domain) / rho * (1 * U.Mpc / U.Myr).to(U.cm / U.s).value
    T_orig = v['temp'][0].reshape(domain)
    fHI = v['ntr_frac_0'][0].reshape(domain)
    p, _, T = rhyme.calc_temperature(v, X=1.0, Y=0.0, Gamma=5./3.)
    T = T[0].reshape(domain)
    p = p.reshape(domain)
    p *= C.m_p.to(U.g).value * (1 * rhyme.dataset.active['h5']['attrs']['pressure_unit']).to(U.cm**-3 * U.cm**2 / U.s**2).value

    M = np.sqrt(vx**2 + vy**2 + vz**2) / np.sqrt(Gamma * p / (C.m_p.to(U.g).value * rho))

    res['n'] = rho
    res['nHI'] = fHI
    res['nHII'] = 1.0 - fHI
    res['T'] = T_orig
    res['p'] = p
    res['M'] = M
    
    return res


def reading_binary_files():
    result = {}
    
    for simname, sim in BINARIES.items():
        for snap_time, snap in sim['orig'].items():
            if snap_time not in result:
                result[snap_time] = {}
                
            if simname not in result[snap_time]:
                result[snap_time][simname] = {}
                
            f = FortranFile(snap['path'], 'r')
            domain = f.read_ints(np.int32)
            
            result[snap_time][simname]['nHI'] = f.read_reals(np.float32).reshape(domain)
            result[snap_time][simname]['p'] = f.read_reals(np.float32).reshape(domain)
            result[snap_time][simname]['T'] = f.read_reals(np.float32).reshape(domain)
            
            f.close()
            
        for snap_time, snap in sim['add'].items():
            if snap_time not in result:
                result[snap_time] = {}
                
            if simname not in result[snap_time]:
                result[snap_time][simname] = {}

            f = FortranFile(snap['path'], 'r')
            domain = f.read_ints(np.int32)
            
            result[snap_time][simname]['n'] = f.read_reals(np.float32).reshape(domain)
            result[snap_time][simname]['M'] = f.read_reals(np.float32).reshape(domain)
            result[snap_time][simname]['nHII'] = f.read_reals(np.float32).reshape(domain)
            
            f.close()
            
    for sn, sim in RHYME.items():
        for t in TIMES:
            r = PyRhyme(sim[0]['path'])
            result[t][sn] = load_rhyme(time_to_snap_id(t, r), r)
        
        r.dataset.close_current()
        r.dataset.clean_all()
        
        del r
    
    return result
            
    
if __name__ == '__main__' and '__file__' not in globals():
    DATA = reading_binary_files()

## Maps <a class="anchor" id="maps"></a>

In [ ]:
def maps(
    t, attrs=['n', 'p', 'T', 'nHI', 'M'],
    titles=['log(<i>n</i>)', 'log(<i>p</i>)', 'log(<i>T</i>)', 'log(f<sub>HI</sub>)', 'log(M)'],
):
    specs = [
        [{"type": "heatmap", "rowspan": 2}, {"type": "heatmap"}, {"type": "heatmap"}, {"type": "heatmap"}],
        [            None                 , {"type": "heatmap"}, {"type": "heatmap"}, {"type": "heatmap"}]
    ] * len(attrs)
    
    vs, hs = 0.012, 0.017
    fig = make_subplots(
        2 * len(attrs), 4,
        column_widths=[2, 1, 1, 1], row_heights=[1] * 2 * len(attrs),
        specs=specs,
        vertical_spacing=vs, horizontal_spacing=hs,
        shared_xaxes=True, shared_yaxes=True,
        x_title='x [pc]', y_title='y [pc]',
    )
    
    setting = {
        'n': {'zmin': -3.8, 'zmax': -1, 'cs': 'Spectral', 'annot_c': 'white',},
        'p': {'zmin': -15.6, 'zmax': -12.6, 'cs': 'Viridis', 'annot_c': 'white',},
        'T': {'zmin': 2, 'zmax': 5, 'cs': 'Inferno', 'annot_c': 'white',},
        'nHI': {'zmin': -5.2, 'zmax': 0, 'cs': 'Spectral', 'annot_c': 'white',},
        'M': {'zmin': -2, 'zmax': 0.5, 'cs': 'RdBu', 'annot_c': 'white',},
    }
    
    for ai, attr in enumerate(attrs):
        for si, (sn, sim) in enumerate(DATA[t].items()):
            if sn == 'Rhyme Case A':
                continue
                
            if sn == 'Rhyme Case B':
                sn = 'Rhyme'

            if sn == 'Rhyme':
                irow, icol, a_ref = 1, 1, 1
            elif sn == 'CAPREOLE+C2-RAY':
                irow, icol, a_ref = 1, 2, 2
            elif sn == 'RSPH':
                irow, icol, a_ref = 1, 3, 3
            elif sn == 'ZEUS-MP':
                irow, icol, a_ref = 1, 4, 4
            elif sn == 'LICORICE':
                irow, icol, a_ref = 2, 2, 5
            elif sn == 'FLASH-HC':
                irow, icol, a_ref = 2, 3, 6
            elif sn == 'CORAL':
                irow, icol, a_ref = 2, 4, 7
            else:
                print('Unknown sim!')
                continue
                
            irow += 2 * ai
                
            shape = sim[attr].shape
            
            if sn == 'ZEUS-MP':
                z = sim[attr][:, int(shape[2]/2), :].transpose()
            else:
                z = sim[attr][int(shape[1]/2), :, :]
                
            fig.add_trace(go.Heatmap(
                z=np.log10(z),
                zauto=False, zmin=setting[attr]['zmin'], zmax=setting[attr]['zmax'],
                colorbar=dict(
                    title=titles[ai], titleside='bottom', titlefont=dict(size=18),
                    thicknessmode='pixels', thickness=10, tickfont=dict(size=18),
                    x=1.02, y=1 - (ai + .5) / len(attrs) - (ai) * vs / 3, len=(.95 - (len(attrs) - 1) * vs) / len(attrs), xanchor='left', yanchor='middle',
                ),
                colorscale=setting[attr]['cs'], showscale=True if si == 0 else False,
            ), irow, icol)
            
            ref = 1 if a_ref == 1 and ai == 0 else f"{a_ref + ai * 7}"
            fig.add_annotation(
                xref=f'x{ref}', yref=f'y{ref}', xanchor='left', yanchor='bottom', x=5, y=3, text=sn,
                showarrow=False, font=dict(color=setting[attr]['annot_c'], size=14),
            )
            
            fig.update_xaxes(
                row=irow, col=icol, mirror=True, title=None,
                titlefont=dict(size=18), tickfont=dict(size=14) if sn == 'Rhyme' else dict(size=11),
                tickmode='array', tickvals=[0, 64, 127], ticktext=['0', '3.3', '6.6'], tickangle=0,
                showticklabels=True if (sn == 'Rhyme' and irow == 2 * len(attrs) - 1) or (irow == 2 * len(attrs)) else False,
            )
            
            fig.update_yaxes(
                row=irow, col=icol, mirror=True, title=None,
                titlefont=dict(size=18), tickfont=dict(size=14),
                tickmode='array', tickvals=[0, 64, 127], ticktext=['0', '3.3', '6.6'],
                showticklabels=True if icol == 1 else False,
            )
            
    width, height = 1000, 1800
            
    fig.update_layout(
        width=width, height=height,
        margin=dict(l=60,r=10,b=60,t=10),
    )

    fig.write_image(f'{FIGURE_DIR}/iliev-7-maps-{int(t):02d}.png', width=width, height=height, scale=3)
    fig.write_image(f'{FIGURE_DIR}/iliev-7-maps-{int(t):02d}.svg', width=width, height=height)
    fig.show()
    

if __name__ == '__main__' and '__file__' not in globals():
    maps(1)
    maps(3)
    maps(10)
    maps(25)
    maps(75)

## Lineouts <a class="anchor" id="lineouts"></a>

In [ ]:
def plot_lineouts(ts, attrs, titles):
    colors = px.colors.sample_colorscale(px.colors.diverging.Portland_r, np.linspace(0, 1, len(DATA[10].keys())))
    
    ticks = {
        'p': {
            'type': 'log',
            'vals': [10**x for x in range(-20, -10)],
            'text': [f"10<sup>{x}</sup>" if x != 0 else '1' for x in range(-20, -10)],
        },
        'T': {
            'type': 'log',
            'vals': [10**x for x in range(2, 8)],
            'text': [f"10<sup>{x}</sup>" if x != 0 else '1' for x in range(2, 8)],
        },
        'fHI': {
            'type': 'log',
            'vals': [10**x for x in range(-10, 1, 2)],
            'text': [f"10<sup>{x}</sup>" if x != 0 else '1' for x in range(-10, 1, 2)],
        },
        'n': {
            'type': 'log',
            'vals': [10**x for x in range(-4, 2)],
            'text': [f"10<sup>{x}</sup>" if x != 0 else '1' for x in range(-4, 2)],
        },
        'M': {
            'type': 'linear',
            'vals': [0, 1, 2, 2.5],
            'text': ['0', '1', '2', '2.5']
        }
    }
    
    len_t = len(ts)
    
    fig = make_subplots(
        len(attrs), len_t,
        column_widths=[1] * len_t, row_heights=[1] * len(attrs),
        vertical_spacing=0.025, horizontal_spacing=0.03,
        shared_xaxes=False, shared_yaxes=True,
        x_title='x / L<sub>box</sub>',
        subplot_titles=[f"t = {t} Myr" for t in ts],
    )
    
    for i in range(len_t):
        fig.layout.annotations[i]["font"] = dict(size=18, color='black')
        fig.layout.annotations[i]["y"] += 0.0125

    fig.layout.annotations[len_t]["font"] = dict(size=18, color='black')
    fig.layout.annotations[len_t]["y"] -= 0.0125
    
    for irow, attr in enumerate(attrs):
        for icol, t in enumerate(ts):
            for si, (sn, sim) in enumerate(DATA[t].items()):
                x = np.linspace(0, 1, len(sim['nHI'][:, 0, 0]))
                if attr == 'fHI':
                    if sn == 'ZEUS-MP':
                        y = np.array(sim['nHI'][:, 64, 64] / (sim['nHI'][:, 64, 64] + sim['nHII'][:, 64, 64]))
                    else:
                        y = np.array(sim['nHI'][64, 64, :] / (sim['nHI'][64, 64, :] + sim['nHII'][64, 64, :]))
                else:
                    if sn == 'ZEUS-MP':
                        y = np.array(sim[attr][:, 64, 64])
                    else:
                        y = np.array(sim[attr][64, 64, :])
                    
                if sn == 'RH1D' and attr == 'p':
                    y /= 10**(-1.35+15.23)
                    
                if sn == 'RH1D' and attr == 'n':
                    y /= 10**(12.50+1.37)
                    
#                 y = np.log10(y)

                fig.add_trace(go.Scatter(
                    x=x, y=y, name=f"{sn}", mode='lines',
                    line=dict(color=colors[si], dash=None, width=1),
                    showlegend=True if irow == icol == 0 else False,
                ), irow+1, icol+1)

                fig.update_xaxes(
                    row=irow+1, col=icol+1, type='linear', mirror=True,
                    titlefont=dict(size=18), tickfont=dict(size=14),
                    tickmode='array', tickvals=[0, 0.5, 1], ticktext=['0', '.5', '1'],
                    showticklabels=True if irow == len(attrs)-1 else False,
                )

                tickvals = list(range(-40, 20, 2))
                ticktext = [f"10<sup>{x}</sup>" if x != 0 else '1' for x in tickvals]

                fig.update_yaxes(
                    row=irow+1, col=icol+1, mirror=True, type=ticks[attr]['type'],
                    title=titles[irow] if icol == 0 else None,
                    titlefont=dict(size=18), tickfont=dict(size=14),
                    tickmode='array', tickvals=ticks[attr]['vals'], ticktext=ticks[attr]['text'],
                    showticklabels=True if icol == 0 else False,
                )
            
    width = 1000
    height = 1250
    
    for i in range(len(attrs)):
        fig.update_yaxes(row=i+1, col=1, titlefont=dict(size=18), tickfont=dict(size=14),)
    
    fig.update_layout(
        width=width, height=height,
        margin=dict(t=60, r=60, b=70, l=80),
        legend=dict(orientation="h", yanchor="top", y=1.12, xanchor="center", x=.5),
    )
    
    fig.write_image(f"{FIGURE_DIR}/iliev-7-lineouts.svg", width=width, height=height)
    fig.write_image(f"{FIGURE_DIR}/iliev-7-lineouts.png", width=width, height=height, scale=3)
    fig.show()

if __name__ == '__main__' and '__file__' not in globals():
    plot_lineouts([1, 3, 10, 25], ['n', 'p', 'T', 'fHI', 'M'], ['n [cm<sup>-3</sup>]', 'p [g / m s<sup>2</sup>]', 'T [K]', 'f<sub>HI</sub>', 'M'])

## I-front position <a class="anchor" id="pos-vel"></a>

In [ ]:
def i_front_get_position(xs, fHI, f0=.5):
    ydir = np.sign(np.diff(fHI))
    turning_points = 1 + np.where(np.diff(ydir) != 0)[0]
    
    signs = np.split(ydir, turning_points)
    xs_grp = np.split(xs, turning_points)
    fHI_grp = np.split(fHI, turning_points)
    fs = [
        (interp1d(y, x, bounds_error=False, fill_value=-1), sgn[0])
        for x, y, sgn in zip(xs_grp, fHI_grp, signs) if len(x) > 2
    ]
    
    return [(float(p[0]), p[1]) for p in [(f[0](f0), f[1]) for f in fs] if p[0] != -1]


def i_front_pos():
    xs = np.linspace(0, 6.6, 128)
    
    for t, snap in DATA.items():
        for sn, sim in DATA[t].items():
            if 'I-front' in sim:
                del sim['I-front']
                
            sim['I-front'] = {}
            
            if sn == 'ZEUS-MP':
                nHI_prof = np.array(sim['nHI'][:, 64, 64])
                nHII_prof = np.array(sim['nHII'][:, 64, 64])
            else:
                nHI_prof = np.array(sim['nHI'][64, 64, :])
                nHII_prof = np.array(sim['nHII'][64, 64, :])

            fHI_prof = nHII_prof / (nHI_prof + nHII_prof)

            pos = i_front_get_position(xs, fHI_prof, f0=0.5)
            if len(pos) == 1:
                sim['I-front']['pos_kpc'] = pos[0][0]
            else:
                sim['I-front']['pos_kpc'] = None
                print(f"{sn}-{t}: {pos}")

if __name__ == '__main__' and '__file__' not in globals():
    i_front_pos()

In [ ]:
def i_front_pos_plot():
    colors = px.colors.sample_colorscale(px.colors.diverging.Portland_r, np.linspace(0, 1, len(DATA[10].keys())))
    
    fig = go.Figure()
    
    times = list(DATA.keys())
    for i, sn in enumerate(DATA[10].keys()):
        ys = [DATA[t][sn]['I-front']['pos_kpc'] for t in times]
        
        fig.add_trace(go.Scatter(
            x=times, y=ys, name=sn,
            mode='lines+markers',
            line=dict(color=colors[i], dash=None, width=1,),
        ))

            
    width, height = 650, 460
            
    fig.update_layout(
        margin=dict(b=10, t=10, l=10, r=10),
        width=width, height=height,
        xaxis=dict(
            type='linear', mirror=True, title='<i>t</i> [Myr]', range=(0, 76),
            titlefont=dict(size=18),
            tickmode='array',
            tickvals=TIMES,
            ticktext=[str(tt) for tt in TIMES],
            tickfont=dict(size=14),
        ),
        yaxis=dict(
            type='linear', mirror=True, title='<i>x</i><sub>I-front</sub> [kpc]',
            titlefont=dict(size=18),
            tickmode='array',
            tickvals=np.linspace(4.5, 6,5, 5),
            ticktext=[f"{yy:2.1f}" for yy in np.linspace(4.5, 6.5, 5)],
            tickfont=dict(size=14),
        ),
        legend=dict(orientation="h", yanchor="top", y=1.25, xanchor="center", x=.5),
    )
    
    fig.write_image(f"{FIGURE_DIR}/iliev-7-I-front-pos.svg", width=width, height=height)
    fig.write_image(f"{FIGURE_DIR}/iliev-7-I-front-pos.png", width=width, height=height, scale=3)   
    fig.show()

if __name__ == '__main__' and '__file__' not in globals():
    i_front_pos_plot()